In [63]:
## load libraries
import os
from dotenv import load_dotenv
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# LangChain core imports
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage

# LangChain specific imports
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Load environment variable
load_dotenv()

False

In [64]:
import json
from pathlib import Path

def load_arxiv_chunks(chunked_path):
    """Load ALL chunked ArXiv papers from data-ingestion pipeline"""
    documents = []
    json_files = sorted(chunked_path.glob("*.json"))
    
    print(f"Loading from {len(json_files)} ArXiv papers...")
    
    for json_file in json_files:
        with open(json_file, 'r') as f:
            chunks = json.load(f)
            for chunk in chunks:
                documents.append({
                    'content': chunk['content'],
                    'metadata': chunk['metadata']
                })
    
    return documents

# Path to hybrid chunked data from data-ingestion pipeline
chunked_data_path = Path("../../data-ingestion/processed/chunked/hybrid/")
arxiv_chunks = load_arxiv_chunks(chunked_data_path)

print(f"\nLoaded {len(arxiv_chunks)} chunks from ArXiv papers")
print(f"\nSample ArXiv paper:")
print(f"   Title: {arxiv_chunks[0]['metadata']['title']}")
print(f"   ArXiv ID: {arxiv_chunks[0]['metadata']['arxiv_id']}")
print(f"   Category: {arxiv_chunks[0]['metadata']['primary_category']}")
print(f"   Section: {arxiv_chunks[0]['metadata']['section']}")

# Convert to LangChain Documents
sample_documents = []
for chunk in arxiv_chunks:
    doc = Document(
        page_content=chunk['content'],
        metadata={
            'arxiv_id': chunk['metadata']['arxiv_id'],
            'title': chunk['metadata']['title'],
            'category': chunk['metadata']['primary_category'],
            'section': chunk['metadata']['section'],
            'chunk_id': chunk['metadata']['chunk_id'],
            'source': 'arxiv'
        }
    )
    sample_documents.append(doc)

print(f"reated {len(sample_documents)} LangChain documents from all 100 papers")

Loading from 15 ArXiv papers...

✅ Loaded 150 chunks from ArXiv papers

Sample ArXiv paper:
   Title: Agentic Test-Time Scaling for WebAgents
   ArXiv ID: 2602.12276v1
   Category: cs.AI
   Section: Abstract
✅ Created 150 LangChain documents


## Load Real ArXiv Research Papers

Loading pre-chunked ArXiv papers from the data-ingestion pipeline to demonstrate FAISS vector store capabilities.

In [65]:
# ArXiv chunks are already optimally chunked from data-ingestion!
# We'll use them directly without additional splitting

chunks = sample_documents
print(f"✅ Using {len(chunks)} pre-chunked ArXiv documents")
print(f"\nExample chunk:")
print(f"Title: {chunks[0].metadata['title'][:60]}...")
print(f"Content: {chunks[0].page_content[:150]}...")
print(f"Metadata: {chunks[0].metadata}")

✅ Using 150 pre-chunked ArXiv documents

Example chunk:
Title: Agentic Test-Time Scaling for WebAgents...
Content: Test-time scaling has become a standard way
to improve performance and boost reliability of
neural network models. However, its behavior
on agentic, m...
Metadata: {'arxiv_id': '2602.12276v1', 'title': 'Agentic Test-Time Scaling for WebAgents', 'category': 'cs.AI', 'section': 'Abstract', 'chunk_id': 0, 'source': 'arxiv'}


In [66]:
print(f"Created {len(chunks)} chunks from {len(sample_documents)} documents")
print("\nExample chunk:")
print(f"Content: {chunks[0].page_content}")
print(f"Metadata: {chunks[0].metadata}")

Created 150 chunks from 150 documents

Example chunk:
Content: Test-time scaling has become a standard way
to improve performance and boost reliability of
neural network models. However, its behavior
on agentic, multi-step tasks remains less well-
understood: small per-step errors can compound
over long horizons; and we find that naive policies
that uniformly increase sampling show diminish-
ing returns. In this work, we present CATTS, a
simple technique for dynamically allocating com-
pute for multi-step agents. We first conduct an
empirical study of inference-time scaling for web
agents. We find that uniformly increasing per-
step compute quickly saturates in long-horizon
environments. We then investigate stronger ag-
gregation strategies, including an LLM-based
Arbiter that can outperform naive voting, but
that can overrule high-consensus decisions. We
show that uncertainty statistics derived from the
agent’s own vote distribution (entropy and top-
1/top-2 margin) correlate with dow

In [67]:
# Initialize HuggingFace embeddings - Free, runs locally!
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

print("HuggingFace Embeddings Loaded!")


HuggingFace Embeddings Loaded!


In [68]:
texts=["AI","MAchine learning","Deep Learning","Neural Network"]
batch_embeddings=embeddings.embed_documents(texts)

In [69]:
### Compare Embedding using cosine similarity

def compare_embeddings(text1:str,text2:str):
    """Compare semantic simialrity of 2 texts usign embeddings"""

    emb1=np.array(embeddings.embed_query(text1))
    emb2=np.array(embeddings.embed_query(text2))

    ## Calculate the simialrity score

    similarity=np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
    return similarity

In [70]:
# Test semantic similarity
print("\nSemantic Similarity Examples:")
print(f"'AI' vs 'Artificial Intelligence': {compare_embeddings('AI', 'Artificial Intelligence'):.3f}")


Semantic Similarity Examples:
'AI' vs 'Artificial Intelligence': 0.791


In [71]:
### Create FAISS Vector Store
vectorstore=FAISS.from_documents(
    documents=chunks,
    embedding=embeddings
)
print(f"Vector store created with {vectorstore.index.ntotal} vectors")

Vector store created with 150 vectors


In [72]:
## Save vector tore for later use
vectorstore.save_local("data/faiss_index")
print("Vector store saved to 'faiss_index' directory")

Vector store saved to 'faiss_index' directory


In [73]:
## load vector store
loaded_vectorstore=FAISS.load_local(
    "data/faiss_index",
    embeddings,
    allow_dangerous_deserialization=True
)

print(f"Loaded vector store contains {loaded_vectorstore.index.ntotal} vectors")

Loaded vector store contains 150 vectors


In [74]:
## Similarity Search on ArXiv Papers
query = "What are neural networks and deep learning architectures?"

results = vectorstore.similarity_search(query, k=3)

print(f"🔍 Query: {query}\n")
print("Top 3 ArXiv papers:\n")
for i, doc in enumerate(results, 1):
    print(f"{i}. {doc.metadata['title'][:65]}...")
    print(f"   ArXiv: {doc.metadata['arxiv_id']} | Section: {doc.metadata['section']}")
    print(f"   Content: {doc.page_content[:150]}...")
    print()

🔍 Query: What are neural networks and deep learning architectures?

Top 3 ArXiv papers:

1. A technical curriculum on language-oriented artificial intelligen...
   ArXiv: 2602.12251v1 | Section: approach
   Content: The technical curriculum on language-oriented AI
in translation and specialised communication is
provided as a GitHub repository2, which consists
of a...

2. Energy-Aware Spike Budgeting for Continual Learning in Spiking Ne...
   ArXiv: 2602.12236v1 | Section: 1
Introduction
   Content: Neuromorphic computing has emerged as a paradigm shift in artificial intelligence, drawing
inspiration from the energy efficiency and temporal dynamic...

3. Energy-Aware Spike Budgeting for Continual Learning in Spiking Ne...
   ArXiv: 2602.12236v1 | Section: methods
   Content: IOP Publishing
Journal vv (yyyy) aaaaaa
Author et al
uncertainty-aware learning, context-aware mechanisms [16] that adaptively modulate network
resour...



In [75]:
### Similarity Search with Scores - Another Query
query2 = "machine learning algorithms and optimization techniques"
results_with_scores = vectorstore.similarity_search_with_score(query2, k=3)

print(f"🔍 Query: {query2}\n")
print("Results with similarity scores:\n")
for i, (doc, score) in enumerate(results_with_scores, 1):
    print(f"{i}. Similarity: {score:.4f}")
    print(f"   Paper: {doc.metadata['title'][:60]}...")
    print(f"   ArXiv: {doc.metadata['arxiv_id']} | Category: {doc.metadata['category']}")
    print(f"   Content: {doc.page_content[:120]}...")
    print()

🔍 Query: machine learning algorithms and optimization techniques

Results with similarity scores:

1. Similarity: 1.2782
   Paper: Mixed-Integer Programming for Change-point Detection...
   ArXiv: 2602.11947v1 | Category: math.OC
   Content: signal of the data, and the term breakpoint to denote the corresponding decision variable in a PWL
model at which model ...

2. Similarity: 1.3002
   Paper: dVoting: Fast Voting for dLLMs...
   ArXiv: 2602.12153v1 | Category: cs.CL
   Content: demonstrate
that our method boosts the performance across all bench-
marks covering mathematical, scientific, and genera...

3. Similarity: 1.3437
   Paper: On the implicit regularization of Langevin dynamics with pro...
   ArXiv: 2602.12257v1 | Category: math.PR
   Content: A central feature of modern machine learning is that models are often heavily over-
parameterized yet still achieve exce...



In [76]:
### Search with Metadata Filtering - Filter by ArXiv category
query3 = "computational methods and algorithms"

# Try to filter by category (if papers have similar categories)
filtered_results = vectorstore.similarity_search(
    query3,
    k=3
)

print(f"🔍 Query: {query3}\n")
for i, doc in enumerate(filtered_results, 1):
    print(f"{i}. {doc.metadata['title'][:65]}...")
    print(f"   Category: {doc.metadata['category']} | Section: {doc.metadata['section']}")
    print(f"   Preview: {doc.page_content[:100]}...")
    print()

🔍 Query: computational methods and algorithms

1. dVoting: Fast Voting for dLLMs...
   Category: cs.CL | Section: results
   Preview: demonstrate
that our method boosts the performance across all bench-
marks covering mathematical, sc...

2. Mixed-Integer Programming for Change-point Detection...
   Category: math.OC | Section: 1
Introduction
   Preview: segment each data point belongs to. Therefore, by explicitly encoding segment assignments, MIP
formu...

3. Mixed-Integer Programming for Change-point Detection...
   Category: math.OC | Section: experiments
   Preview: perspective, the key combinatorial decision concerns the optimal partitioning of data points into
co...



In [77]:
## LLM - HuggingFace Model (Chat-based)
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

# Set HuggingFace API token from .env
huggingface_token = os.getenv("HUGGINGFACE_API_KEY")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = huggingface_token

# Initialize HuggingFace LLM via chat endpoint
endpoint = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    max_new_tokens=512,
    temperature=0.7,
    huggingfacehub_api_token=huggingface_token,
)

llm = ChatHuggingFace(llm=endpoint)

print("✅ HuggingFace LLM Loaded!")
print(f"   Model: HuggingFaceH4/zephyr-7b-beta")

✅ HuggingFace LLM Loaded!
   Model: HuggingFaceH4/zephyr-7b-beta


In [78]:
# Test the LLM
response = llm.invoke("What is AI?")
print("LLM Test Response:")
print(response.content)

LLM Test Response:
AI, or artificial intelligence, refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning, reasoning, and decision-making. It involves the use of sophisticated algorithms and machine learning techniques to enable computers to process and analyze large amounts of data, identify patterns, and make informed decisions or suggestions based on that data. AI systems can adapt and learn over time, improving their performance as they are exposed to more information, without being explicitly programmed. AI has the potential to revolutionize various industries and improve efficiency, accuracy, and productivity in various applications, such as healthcare, finance, transportation, and manufacturing, among others. It encompasses various subfields like natural language processing, computer vision, robotics, and machine learning, which enable computers to understand and interpret human language, recognize objects

In [79]:
# 1. Simple RAG Chain with LCEL
simple_prompt = ChatPromptTemplate.from_template("""Answer the question based only on the following context:
Context: {context}

Question: {question}

Answer:""")

In [80]:
## Basic retriever
retriever=vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":3}
)

In [81]:
from typing import List
# Format documents for the prompt
def format_docs(docs: List[Document]) -> str:
    """Format documents for insertion into prompt"""
    formatted = []
    for i, doc in enumerate(docs):
        source = doc.metadata.get('source', 'Unknown')
        formatted.append(f"Document {i+1} (Source: {source}):\n{doc.page_content}")
    return "\n\n".join(formatted)

In [82]:
simple_rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | simple_prompt
    | llm
    | StrOutputParser()
)

In [83]:
### Conversational RAg Chain

conversational_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the provided context to answer questions."),
    ("placeholder", "{chat_history}"),
    ("human", "Context: {context}\n\nQuestion: {input}"),
])

In [84]:
def create_conversational_rag():
    """Create a conversational RAG chain with memory"""
    return (
        RunnablePassthrough.assign(
            context=lambda x: format_docs(retriever.invoke(x["input"]))
        )
        | conversational_prompt
        | llm
        | StrOutputParser()
    )

conversational_rag = create_conversational_rag()

In [85]:
### streaming RAG chain
streaming_rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | simple_prompt
    | llm
    | StrOutputParser()
)

print("RAG chains created successfully!")
print("Available chains:")
print("- simple_rag_chain: Basic Q&A")
print("- conversational_rag: Maintains conversation history")
print("- streaming_rag_chain: Supports token streaming")

RAG chains created successfully!
Available chains:
- simple_rag_chain: Basic Q&A
- conversational_rag: Maintains conversation history
- streaming_rag_chain: Supports token streaming


In [86]:
# Test function for different chain types
def test_rag_chains(question: str):
    """Test all RAG chain variants"""
    print(f"Question: {question}")
    print("=" * 80)
    
    # 1. Simple RAG
    print("\n1. Simple RAG Chain:")
    answer = simple_rag_chain.invoke(question)
    print(f"Answer: {answer}")

    # 2. Streaming RAG
    print("\n2. Streaming RAG:")
    print("Answer: ", end="", flush=True)
    for chunk in streaming_rag_chain.stream(question):
        print(chunk, end="", flush=True)
    print()

In [87]:
test_rag_chains("What is the difference between AI and machine learning")

Question: What is the difference between AI and machine learning

1. Simple RAG Chain:
Answer: Based on the provided context, it is clear that there is a distinction between AI and machine learning. While AI technologies such as LLMs (Language Language Models) and NMT (Neural Machine Translation) are being widely adopted in various industries due to their practical applications, there is a call for promoting technical AI literacy among industry stakeholders to better understand their underlying principles rather than just relying on their performance. This is in contrast to relying solely on their results. The importance of technical AI literacy is emphasized by authors such as Doherty and Kenny in the context of SMT (Statistical Machine Translation) and neural machine translation (NMT) to dispel the mystery surrounding these technologies. Additionally, Shneiderman suggests that industry stakeholders require a deeper understanding of AI in general, specifically in the context of langua

In [88]:
# Test with multiple questions
test_questions = [
    "What is the difference between AI and Machine Learning?",
    "Explain deep learning in simple terms",
    "How does NLP work?"
]

for question in test_questions:
    print("\n" + "=" * 80 + "\n")
    test_rag_chains(question)



Question: What is the difference between AI and Machine Learning?

1. Simple RAG Chain:
Answer: Based on the given context, there seems to be no direct question asking about the difference between AI and machine learning. However, the term "technical AI literacy" is used interchangeably with "machine learning" in Document 1, suggesting that AI and machine learning may be used synonymously in that context. Therefore, it can be inferred that AI and machine learning are referring to the same concept in this case. Therefore, there is no clear distinction between the two in this specific context. However, in Document 2, the term "AI literacy" is further broken down into subcategories such as technical, performance-focused, interaction-focused, and ethical/societal literacy, implying that AI encompasses more than just the technical aspects of machine learning. Therefore, it can be inferred that AI may go beyond just machine learning, as those subcategories suggest broader concepts related 

In [89]:
## Conversational example
print("\n3. Conversational RAG Example:")
chat_history = []

# First question
q1 = "What is machine learning?"
a1 = conversational_rag.invoke({
    "input": q1,
    "chat_history": chat_history
})

print(f"Q1: {q1}")
print(f"A1: {a1}")


3. Conversational RAG Example:
Q1: What is machine learning?
A1: 
Based on the context provided, machine learning is a type of artificial intelligence technique that involves heavily parameterized models with over-fitting capabilities, which can achieve excellent generalization performance even when there are fewer parameters than training data points. This is in contrast to classical statistical theory that suggests the need for regularization to prevent overfitting in heavily parameterized models. This concept has been observed in practice, as explained in Document 2. Additionally, in Document 1, the use of machine learning systems is praised for demystifying their black-box nature, particularly in the contexts of language translation and neural machine translation. There is a call for fostering technical AI literacy among industry stakeholders to better understand these technologies beyond just their applications, as argued by Doherty and Kenny in statistical machine translation an

In [90]:
# Update history
chat_history.extend([
    HumanMessage(content=q1),
    AIMessage(content=a1)
])

In [91]:
# Follow-up question
q2 = "How is it different from traditional programming?"
a2 = conversational_rag.invoke({
    "input": q2,
    "chat_history": chat_history
})
print(f"\nQ2: {q2}")
print(f"A2: {a2}")


Q2: How is it different from traditional programming?
A2: 
Based on the provided context, machine learning differs from traditional programming in that it involves heavily parameterized models with the ability to achieve excellent generalization performance even when there are fewer training data points through over-fitting, unlike traditional programming techniques that may require regularization to prevent overfitting due to the need for fewer parameters. This concept is explained in Document 2. Additionally, machine learning systems are being increasingly adopted in language translation and neural machine translation (NMT), as discussed in Documents 1 and 3. While traditional programming focuses on writing explicit instructions for computers to perform specific tasks, machine learning allows computers to learn and make predictions or decisions based on patterns and inputs without explicit instructions, relying on large amounts of data to improve their performance over time through 